In [37]:
import polars as pl

In [38]:
df = pl.DataFrame({
    "compañia": ["A", "B", "A", "C"],
    "valor": [100, 200, 300, 400]
})
df.head()

compañia,valor
str,i64
"""A""",100
"""B""",200
"""A""",300
"""C""",400


In [39]:
cias = df["compañia"].unique().to_list()
print(cias)

['C', 'A', 'B']


In [40]:
for cia in cias:
    subset = df.filter(pl.col("compañia") == cia)
    print(f"{cia} tiene {subset.shape[0]} movimientos")

C tiene 1 movimientos
A tiene 2 movimientos
B tiene 1 movimientos


CARGA DIFERIDA

In [41]:
lz_mov = pl.scan_csv("movimientos.csv")
print(lz_mov)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

Csv SCAN [movimientos.csv] [id: 5694557392608]
PROJECT */5 COLUMNS


SELECCION DE COLUMNAS AVANZADAS

In [42]:
sel1 = lz_mov.select(["NUM_ID_CIA", "VALOR"])
sel1.collect().head()

NUM_ID_CIA,VALOR
i64,i64
1,250000
1,180000
2,320000
2,150000
1,500000


Seleccionar por patrón (columnas que contienen cierto texto)

In [43]:
sel2 = lz_mov.select(pl.col("^STR_.*$")) #EMPIEZA CON STR
sel2.collect().head()

STR_CUENTA,STR_CENTRO_COSTO
i64,str
510505,"""CC001"""
510506,"""CC002"""
410101,"""CC003"""
410102,"""CC004"""
510507,"""CC005"""


Seleccionar y crear columnas al mismo tiempo

In [44]:
sel3 = lz_mov.select([
    pl.col("NUM_ID_CIA"),
    (pl.col("VALOR") / 1000).alias("valor_en_miles")
])
sel3.collect().head()

NUM_ID_CIA,valor_en_miles
i64,f64
1,250.0
1,180.0
2,320.0
2,150.0
1,500.0


4️⃣ **Filtrado con múltiples condiciones**

AND (&)

In [45]:
filtro_and = lz_mov.filter(
    (pl.col("VALOR") < 200000) & (pl.col("CIUDAD") == "CALI")
)
filtro_and.collect().head()

NUM_ID_CIA,STR_CUENTA,STR_CENTRO_COSTO,VALOR,CIUDAD
i64,i64,str,i64,str
2,410102,"""CC004""",150000,"""CALI"""


OR (|)

In [46]:
filtro_or = lz_mov.filter(
    (pl.col("CIUDAD") == "CLAI") | (pl.col("CIUDAD") == "BOGOTA")
)
filtro_or.collect().head()

NUM_ID_CIA,STR_CUENTA,STR_CENTRO_COSTO,VALOR,CIUDAD
i64,i64,str,i64,str


NOT (~)

In [47]:
filtro_not = lz_mov.filter(
    ~(pl.col("CIUDAD") == "CALI")
)
filtro_not.collect().head()

NUM_ID_CIA,STR_CUENTA,STR_CENTRO_COSTO,VALOR,CIUDAD
i64,i64,str,i64,str
1,510506,"""CC002""",180000,"""MEDELLÍN"""
2,410101,"""CC003""",320000,"""BOGOTÁ"""


5️⃣ **Filtrado con funciones y expresiones**

In [48]:
ciudades = ["CALI", "MEDELLÍN"]
filtro_in = lz_mov.filter(
    (pl.col("CIUDAD").is_in(ciudades))
)
filtro_in.collect().head()

NUM_ID_CIA,STR_CUENTA,STR_CENTRO_COSTO,VALOR,CIUDAD
i64,i64,str,i64,str
1,510505,"""CC001""",250000,"""CALI"""
1,510506,"""CC002""",180000,"""MEDELLÍN"""
2,410102,"""CC004""",150000,"""CALI"""
1,510507,"""CC005""",500000,"""CALI"""


Filtrar por longitud de texto

In [50]:
filtro_len = lz_mov.with_columns(
    pl.col("STR_CUENTA").cast(pl.Utf8)
).filter(
    pl.col("STR_CUENTA").str.len_chars() == 6
)
print(filtro_len.collect())

shape: (5, 5)
┌────────────┬────────────┬──────────────────┬────────┬──────────┐
│ NUM_ID_CIA ┆ STR_CUENTA ┆ STR_CENTRO_COSTO ┆ VALOR  ┆ CIUDAD   │
│ ---        ┆ ---        ┆ ---              ┆ ---    ┆ ---      │
│ i64        ┆ str        ┆ str              ┆ i64    ┆ str      │
╞════════════╪════════════╪══════════════════╪════════╪══════════╡
│ 1          ┆ 510505     ┆ CC001            ┆ 250000 ┆ CALI     │
│ 1          ┆ 510506     ┆ CC002            ┆ 180000 ┆ MEDELLÍN │
│ 2          ┆ 410101     ┆ CC003            ┆ 320000 ┆ BOGOTÁ   │
│ 2          ┆ 410102     ┆ CC004            ┆ 150000 ┆ CALI     │
│ 1          ┆ 510507     ┆ CC005            ┆ 500000 ┆ CALI     │
└────────────┴────────────┴──────────────────┴────────┴──────────┘


6️⃣ **Selección dinámica de columnas**

In [51]:
solo_numericas = lz_mov.select(pl.col(pl.NUMERIC_DTYPES))
solo_numericas.collect().head()

C:\Users\Nelson.Daza\AppData\Local\Temp\ipykernel_8976\3792018676.py:1: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  solo_numericas = lz_mov.select(pl.col(pl.NUMERIC_DTYPES))


NUM_ID_CIA,STR_CUENTA,VALOR
i64,i64,i64
1,510505,250000
1,510506,180000
2,410101,320000
2,410102,150000
1,510507,500000


SELECCIONAR TODAD LAS COLUMNAS MENOS UNA

In [52]:
sin_centro = lz_mov.select(pl.all().exclude("STR_CENTRO_COSTO"))
sin_centro.collect().head()

NUM_ID_CIA,STR_CUENTA,VALOR,CIUDAD
i64,i64,i64,str
1,510505,250000,"""CALI"""
1,510506,180000,"""MEDELLÍN"""
2,410101,320000,"""BOGOTÁ"""
2,410102,150000,"""CALI"""
1,510507,500000,"""CALI"""
